## Importing Relevant Libraries

In [6]:
# installing relevant libaries from the terminal:

# pip install google-cloud-bigquery 
# pip install google-cloud-bigquery-storage 
# pip install db-dtypes
# pip install pandasql
# pip install pandas-gbq 

In [2]:
# importing relevant libaries

from google.cloud import bigquery # to connect to the BigQuery client
from google.cloud import bigquery_storage # for efficient data loading and extraction compared to the traditional REST API
import db_dtypes # to handle the conversion between database-specific data types and their corresponding Pandas data types
from pandasql import sqldf # to use SQL on python dataframes
from pandas_gbq import to_gbq # to write data to BigQuery tables

## Reading Data from BigQuery

- First we'll need to authenticate our Python application with Google Cloud Platform (GCP). For that, we'll create a service account in the GCP console, download its JSON key file, and use it in oour Python code.
- We use the code below to interact with the BigQuery API and use the service account JSON key file to creates a BigQuery client object.

In [4]:
# entering the 'path/to/your/key.json' with the actual path to your JSON key file
client = bigquery.Client.from_service_account_json('JSON_KEY.json')

In [5]:
# now we can query and import data from our BigQuery tables

query = """
SELECT *
FROM company_source_layer.orders_source
"""

query_job = client.query(query)
results = query_job.result()

df = results.to_dataframe()

In [6]:
# view the results

df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,6231,CA-2014-127656,2014-07-12,2014-07-18,Standard Class,NW-18400,Natalie Webber,Consumer,United States,Waterloo,...,50701,Central,OFF-AR-10001166,Office Supplies,Art,Staples,30.32,4,0.0,11.8248
1,4190,CA-2013-157714,2013-09-27,2013-10-02,Second Class,CS-12175,Charles Sheldon,Corporate,United States,Iowa City,...,52240,Central,OFF-PA-10004022,Office Supplies,Paper,Hammermill Color Copier Paper (28Lb. and 96 Br...,9.99,1,0.0,4.4955
2,3426,CA-2012-153381,2012-09-24,2012-09-28,Standard Class,DE-13255,Deanra Eno,Home Office,United States,Dubuque,...,52001,Central,OFF-BI-10001525,Office Supplies,Binders,"Acco Pressboard Covers with Storage Hooks, 14 ...",15.24,4,0.0,6.8580
3,3427,CA-2012-153381,2012-09-24,2012-09-28,Standard Class,DE-13255,Deanra Eno,Home Office,United States,Dubuque,...,52001,Central,FUR-CH-10000988,Furniture,Chairs,Hon Olson Stacker Stools,1408.10,10,0.0,394.2680
4,8001,US-2012-151407,2012-11-08,2012-11-12,Standard Class,RD-19585,Rob Dowd,Consumer,United States,Dubuque,...,52001,Central,TEC-PH-10003885,Technology,Phones,Cisco SPA508G,263.96,4,0.0,76.5484


## Calculating Key Monthly Metrics 

In [50]:
# using the raw orders data to compute some key metrics on a monthly level

query = """
SELECT
  DATE(strftime('%Y', `Order Date`) || '-' || strftime('%m', `Order Date`) || '-' || '01') AS month,
  COUNT(*) AS num_of_rows,
  COUNT(DISTINCT `Order ID`) AS num_of_orders,
  ROUND(SUM(`Sales`)) AS revenue,
  ROUND(SUM(`Profit`)) AS profit,
  ROUND(SUM(`Sales`) / COUNT(DISTINCT `Order ID`)) AS avg_order_value,
  COUNT(DISTINCT `Customer ID`) AS active_customers,
  ROUND(COUNT(DISTINCT `Order ID`) / COUNT(DISTINCT `Customer ID`), 1) AS orders_per_active_customer,
  ROUND(SUM(`Sales`) / COUNT(DISTINCT `Customer ID`)) AS revenue_per_active_customer,
  ROUND(SUM(`Profit`) / COUNT(DISTINCT `Customer ID`)) AS profit_per_active_customer

FROM df
GROUP BY month
ORDER BY month DESC
"""

monthly_metrics = sqldf(query)
monthly_metrics.head()

,month,num_of_rows,num_of_orders,revenue,profit,avg_order_value,active_customers,orders_per_active_customer,revenue_per_active_customer,profit_per_active_customer
0,2014-12-01,477,235,90475.0,8533.0,385.0,203,1.0,446.0,42.0
1,2014-11-01,447,252,112326.0,9683.0,446.0,211,1.0,532.0,46.0
2,2014-10-01,302,150,77794.0,9441.0,519.0,139,1.0,560.0,68.0
3,2014-09-01,463,229,90489.0,11395.0,395.0,200,1.0,452.0,57.0
4,2014-08-01,218,110,61516.0,8894.0,559.0,103,1.0,597.0,86.0


## Writing Data to BigQuery

In [57]:
# method 1

monthly_metrics.to_gbq(
    destination_table = 'company_reporting_layer.monthly_metrics',
    project_id = client.project, # or your actual 'your_project_id'
    if_exists = 'replace',
    # if_exists = 'replace' --> Replace existing table
    # if_exists = 'append' --> append data to existing table
    # Optional: Specify table_schema if needed, otherwise the function with infer the schema itself
    table_schema = [ 
        {'name': 'month', 'type': 'DATE', 'mode': 'REQUIRED'}, # 'mode' can be 'NULLABLE' where NULL values are accepted
        {'name': 'num_of_rows', 'type': 'INTEGER', 'mode': 'REQUIRED'},
        {'name': 'num_of_orders', 'type': 'INTEGER', 'mode': 'REQUIRED'},
        {'name': 'revenue', 'type': 'FLOAT64', 'mode': 'REQUIRED'},
        {'name': 'profit', 'type': 'FLOAT64', 'mode': 'REQUIRED'},
        {'name': 'avg_order_value', 'type': 'FLOAT64', 'mode': 'REQUIRED'},
        {'name': 'active_customers', 'type': 'INTEGER', 'mode': 'REQUIRED'},
        {'name': 'orders_per_active_customer', 'type': 'FLOAT64', 'mode': 'REQUIRED'},
        {'name': 'revenue_per_active_customer', 'type': 'FLOAT64', 'mode': 'REQUIRED'},
        {'name': 'profit_per_active_customer', 'type': 'FLOAT64', 'mode': 'REQUIRED'},
    ]  
)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


In [59]:
# method 2

to_gbq(
    dataframe = monthly_metrics, 
    destination_table = 'company_reporting_layer.monthly_metrics_2', 
    project_id = client.project, 
    if_exists='replace',
    table_schema = [ 
        {'name': 'month', 'type': 'DATE', 'mode': 'REQUIRED'},
        {'name': 'num_of_rows', 'type': 'INTEGER', 'mode': 'REQUIRED'},
        {'name': 'num_of_orders', 'type': 'INTEGER', 'mode': 'REQUIRED'},
        {'name': 'revenue', 'type': 'FLOAT64', 'mode': 'REQUIRED'},
        {'name': 'profit', 'type': 'FLOAT64', 'mode': 'REQUIRED'},
        {'name': 'avg_order_value', 'type': 'FLOAT64', 'mode': 'REQUIRED'},
        {'name': 'active_customers', 'type': 'INTEGER', 'mode': 'REQUIRED'},
        {'name': 'orders_per_active_customer', 'type': 'FLOAT64', 'mode': 'REQUIRED'},
        {'name': 'revenue_per_active_customer', 'type': 'FLOAT64', 'mode': 'REQUIRED'},
        {'name': 'profit_per_active_customer', 'type': 'FLOAT64', 'mode': 'REQUIRED'},
    ]  
)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]
